In [37]:
import sqlite3
import pandas as pd

In [38]:
database = r'C:\Users\nnoar\OneDrive\Desktop\New folder\basketball.sqlite'

In [39]:
conn = sqlite3.connect(database)

In [40]:
tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)

In [41]:
tables

,type,name,tbl_name,rootpage,sql
0,table,Player,Player,2,"CREATE TABLE ""Player"" (\n""id"" TEXT,\n ""full_n..."
1,table,Team,Team,4,"CREATE TABLE ""Team"" (\n""id"" TEXT,\n ""full_nam..."
2,table,Team_Attributes,Team_Attributes,5,"CREATE TABLE ""Team_Attributes"" (\n""ID"" TEXT,\n..."
3,table,Team_History,Team_History,6,"CREATE TABLE ""Team_History"" (\n""ID"" TEXT,\n ""..."
4,table,Player_Attributes,Player_Attributes,15,"CREATE TABLE ""Player_Attributes"" (\n""ID"" TEXT,..."
5,table,Game_Officials,Game_Officials,8375,"CREATE TABLE ""Game_Officials"" (\n""OFFICIAL_ID""..."
6,table,Game_Inactive_Players,Game_Inactive_Players,8997,"CREATE TABLE ""Game_Inactive_Players"" (\n""PLAYE..."
7,table,Team_Salary,Team_Salary,13415,CREATE TABLE `Team_Salary` (\n `nameTeam` TEX...
8,table,Player_Salary,Player_Salary,23558,CREATE TABLE `Player_Salary` (\n `slugSeason`...
9,table,Draft,Draft,23559,"CREATE TABLE `Draft` (\n `yearDraft` REAL,\n ..."


Pull Active Players from the Players Table

In [46]:
active = pd.read_sql("""SELECT *
                        FROM Player
                        WHERE is_active =1;""", conn)
active = active[['id','full_name']]
active.head()

,id,full_name
0,203500,Steven Adams
1,1628389,Bam Adebayo
2,200746,LaMarcus Aldridge
3,1629734,Kyle Alexander
4,1629638,Nickeil Alexander-Walker


Pull 2020-2021 players salaries

In [55]:
salaries_21 = pd.read_sql("""SELECT * 
                            FROM Player_Salary
                            WHERE slugSeason = "2020-21";""",conn)
salaries_21 = salaries_21[['nameTeam','namePlayer','value']]
salaries_21 = salaries_21.rename(columns={'nameTeam':'Team','namePlayer':'full_name'})
salaries_21.head()
                            


,Team,full_name,value
0,Atlanta Hawks,Bogdan Bogdanovic,18000000.0
1,Atlanta Hawks,Brandon Goodwin,1701593.0
2,Atlanta Hawks,Bruno Fernando,1517981.0
3,Atlanta Hawks,Cam Reddish,4458000.0
4,Atlanta Hawks,Clint Capela,16000000.0


Pull draft positions of each player and merge with active players

In [66]:
draft = pd.read_sql("""SELECT *
                        FROM Draft;""",conn)
draft = draft[['yearDraft','numberPickOverall','numberRound','namePlayer','idPlayer','nameTeam']]
draft = draft.rename(columns={'yearDraft':'Year_Drafted','numberPickOverall':'Overall_Pick','numberRound':'Round','namePlayer':'full_name','idPlayer':'id','nameTeam':'Drafted_by'})
active_info = pd.merge(active,draft, on='full_name')
active_info.head()

,id_x,full_name,Year_Drafted,Overall_Pick,Round,id_y,Drafted_by
0,203500,Steven Adams,2013.0,12.0,1.0,203500.0,Oklahoma City Thunder
1,1628389,Bam Adebayo,2017.0,14.0,1.0,1628389.0,Miami Heat
2,200746,LaMarcus Aldridge,2006.0,2.0,1.0,200746.0,Chicago Bulls
3,1629638,Nickeil Alexander-Walker,2019.0,17.0,1.0,1629638.0,Brooklyn Nets
4,1628960,Grayson Allen,2018.0,21.0,1.0,1628960.0,Utah Jazz


In [69]:
active_info.drop(['id_y'], axis=1)
active_info = active_info.rename(columns={'id_x':'id'})
